In [52]:
from sqlalchemy import * 
from DBEngines import engineName ### local file
from sqlalchemy.sql import table, column, select, update, insert
from sqlalchemy.orm import sessionmaker,mapper
import pandas as pd
import time
import datetime as dt
from datetime import timedelta,datetime,date
from functools import reduce
import sys
import json
import numpy as np
import traceback
from ErrorMail import sendmail  ### local file
from FuncTimer import fn_timer  ### local file
from retry import retry

engineShiXiong06 = engineName('engineShiXiong06')
engineShopstat = engineName('engineShopstat')

In [53]:
#@fn_timer
## 4-10 14:00 
timeRange = [1523340000,1523345760]
sql = 'select id,time,app,page,os,uv_count,uv_total_hour,uv_total_day,uv_total_week,uv_total_month from shop_view where time>={i[0]} and time<{i[1]} ;'.format(i=timeRange)
print(sql)
dataInTestDB = pd.read_sql(sql,engineShiXiong06,chunksize=100000)

select id,time,app,page,os,uv_count,uv_total_hour,uv_total_day,uv_total_week,uv_total_month from shop_view where time>=1523340000 and time<1523345760 ;
2018-04-10 15:41:59,789 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2018-04-10 15:41:59,790 INFO sqlalchemy.engine.base.Engine {}
2018-04-10 15:41:59,796 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2018-04-10 15:41:59,797 INFO sqlalchemy.engine.base.Engine {}
2018-04-10 15:41:59,799 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2018-04-10 15:41:59,800 INFO sqlalchemy.engine.base.Engine {}
2018-04-10 15:41:59,803 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2018-04-10 15:41:59,804 INFO sqlalchemy.engine.base.Engine {}
2018-04-10 15:41:59,806 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2018-04-10 15:41:59,806 INFO sqlalchemy.engine.base.Engine {}
2018-04-10 

In [54]:
## 找ID
sql = 'select id,time,app,page,os from shop_view where time>={i[0]} and time<{i[1]};'.format(i=timeRange)
print(sql)
IDtestDB = pd.read_sql(sql,engineShopstat)

select id,time,app,page,os from shop_view where time>=1523340000 and time<1523345760;


In [55]:
## update ,需要先找出在线上库中的 id， 匹配后再update
@fn_timer
def updateToMysql(finalDataDF):
    class ShopView(object):
        def __init__(self,id,uv_total_day,uv_total_week,uv_total_month,uv_total_hour,uv_count):
            
            self.id=id
            self.uv_total_day=uv_total_day
            self.uv_total_week=uv_total_week
            self.uv_total_month=uv_total_month
            self.uv_total_hour=uv_total_hour
            self.uv_count=uv_count

    metadata = MetaData(bind = engineShopstat)
    shop_view = Table('shop_view', metadata, autoload=True)
    mapper(ShopView,shop_view)
    
    Session = sessionmaker(bind = engineShopstat)
    session = Session()
    
    finalDataDict=finalDataDF.to_dict('record')
    print('update begins')
    session.bulk_update_mappings(ShopView,finalDataDict)
    session.commit()
    session.close()
    
    print("%d records updated over~" %len(finalDataDict))

In [56]:
### update
i=0
for chunk in dataInTestDB:
    chunk.drop('id',axis=1,inplace=True)
    chunk.fillna(0,inplace=True)
    datas= pd.merge(chunk,IDtestDB,how='left',on=['time','app','os','page'])  
    datas.drop(['time','app','os','page'],axis=1,inplace=True)
    cols = ['id','uv_count','uv_total_hour','uv_total_day','uv_total_week','uv_total_month']
    for col in cols:
        datas[col]=datas[col].map(lambda x:str(int(x)))
        
    #datas[cols] = datas[cols].apply(pd.to_numeric, downcast='unsigned', axis=1)
    print(i,datas[:2])
    
    updateToMysql(datas)
    print(len(datas),'updated over')
    
    i+=len(datas)
print('all updated over, %d 条' %i)


0   uv_count uv_total_hour uv_total_day uv_total_week uv_total_month       id
0     3822         72316       526467             0              0  8049941
1     2485         41775       309087             0              0  8049942
update begins
7296 records updated over~

函数 updateToMysql 运行用时: 16.83657670021057 seconds
7296 updated over
all updated over, 7296 条


In [ ]:
### insert
i=0
for chunk in dataInTestDB:
    chunk.to_sql(name='shop_view',con=engineShopstat,if_exists='append',index=False)
    print(len(chunk),'insert over')
    i+=len(chunk)
print('all insert over, %d 条' %i)

In [9]:
0.6*100*1440/(3600)

24.0

In [10]:
389*12/3600

1.2966666666666666

In [41]:
#@fn_timer

## 4/1的数load回测试库
timeRange = [1522512000,1522598400]
sql = 'select * from shop_view where time>={i[0]} and time<{i[1]} ;'.format(i=timeRange)
print(sql)
dataInOnline = pd.read_sql(sql,engineShopstat)

select * from shop_view where time>=1522512000 and time<1522598400 ;


ValueError: labels ['id'] not contained in axis

In [4]:
## update ,需要先找出在线上库中的 id， 匹配后再update
@fn_timer
def updateToMysql(finalDataDF):
    class ShopView(object):
        def __init__(self,id,time,app,os,page,uv_total_day,uv_total_week,uv_total_month,uv_total_hour,uv_count,\
                     pv_count,ud_youpin_newuser,uv_new_total_week,uv_new_total_day,uv_new_count,uv_slient,\
                     uv_total_live1_day,uv_total_live3_day,uv_total_live7_day,submit_count,so_count,su_count,submit_gcount,\
                     su_total_month,su_total_week,su_total_day,su_total_hour,sale_count,po_count,pu_total_month,pu_total_week,\
                     pu_total_day,pu_total_hour,pu_count,pay_gcount,pun_total,pun_count,per_pay_day,per_pay_week,per_trans_day,\
                     per_trans_week,sales_day,sales_week,sales_month,sales_year,submit_sales_day,submit_sales_week,submit_sales_month,\
                     submit_sales_year):
            
            self.id=id
            self.time=time
            self.app=app
            self.os=os
            self.page=page
            self.uv_total_day=uv_total_day
            self.uv_total_week=uv_total_week
            self.uv_total_month=uv_total_month
            self.uv_total_hour=uv_total_hour
            self.uv_count=uv_count
            self.pv_count=pv_count
            self.ud_youpin_newuser=ud_youpin_newuser
            self.uv_new_total_week=uv_new_total_week
            self.uv_new_total_day=uv_new_total_day
            self.uv_new_count=uv_new_count
            self.uv_slient=uv_slient
            self.uv_total_live1_day=uv_total_live1_day
            self.uv_total_live3_day=uv_total_live3_day
            self.uv_total_live7_day=uv_total_live7_day
            self.submit_count=submit_count
            self.so_count=so_count
            self.su_count=su_count
            self.submit_gcount=submit_gcount
            self.su_total_month=su_total_month
            self.su_total_week=su_total_week
            self.su_total_day=su_total_day
            self.su_total_hour=su_total_hour
            self.sale_count=sale_count
            self.po_count=po_count
            self.pu_total_month=pu_total_month
            self.pu_total_week=pu_total_week
            self.pu_total_day=pu_total_day
            self.pu_total_hour=pu_total_hour
            self.pu_count=pu_count
            self.pay_gcount=pay_gcount
            self.pun_total=pun_total
            self.pun_count=pun_count
            self.per_pay_day=per_pay_day
            self.per_pay_week=per_pay_week
            self.per_trans_day=per_trans_day
            self.per_trans_week=per_trans_week
            self.sales_day=sales_day
            self.sales_week=sales_week
            self.sales_month=sales_month
            self.sales_year=sales_year
            self.submit_sales_day=submit_sales_day
            self.submit_sales_week=submit_sales_week
            self.submit_sales_month=submit_sales_month
            self.submit_sales_year=submit_sales_year

    metadata = MetaData(bind = engineShiXiong06)
    shop_view = Table('shop_view', metadata, autoload=True)
    mapper(ShopView,shop_view)
    
    Session = sessionmaker(bind = engineShiXiong06)
    session = Session()
    
    finalDataDict=finalDataDF.to_dict('record') 
    session.bulk_update_mappings(ShopView,finalDataDict)
    session.commit()
    session.close()
    
    print("%d records updated over~" %len(finalDataDict))

In [50]:
dataToTestDB = pd.merge(dataInOnline,IDtestDB,how='right',on=['time','app','os','page']) 

In [54]:

updateToMysql(dataToTestDB)


2018-04-03 19:09:42,069 INFO sqlalchemy.engine.base.Engine SHOW CREATE TABLE `shop_view`
2018-04-03 19:09:42,072 INFO sqlalchemy.engine.base.Engine {}
109440 records updated over~

函数 updateToMysql 运行用时: 6.569838523864746 seconds


In [3]:
3600*24*30

2592000